In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [ ]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c, Corredor__c,Pa_s_del_Paciente__c, Habilitado__c from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "Vigencia__c",
        "Corredor__c",
        "Pa_s_del_Paciente__c",
        "Habilitado__c",
    ]
]
Empresas_SF.shape


In [ ]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape


In [ ]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

In [ ]:
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Empleador__c": "Empleador",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
        "Vigencia__c": "Vigente",
        "Corredor__c": "Corredor",
        "Pa_s_del_Paciente__c": "Pais",
        "Habilitado__c": "Habilitado",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Empleador",
        "Holding",
        "Campaña",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
        "Habilitado",
    ]
]

In [ ]:
Programas_empresas.to_excel("C:/Users/jmartinez/Documents/Empresas2.xlsx", index=False)
# .to_excel("F:/Empresas SF/reportes/empresas2.xlsx", index=False)
Programas_empresas.head()

Finalizacion de Recorrido por las empresas

Ahora comienza la recoleccion de las fichas y sus datos


In [ ]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c,Empresa__c,Empresa_Principal__c from ServiceAppointment
    """
)
Citas_de_servicio = pd.DataFrame(results["records"])
# itas_de_servicio["Empresas"] = np.nan
# Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name'])
# Citas_de_servicio["Empresas Id"] = np.nan
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",
        "RUT__c",
        "CreatedDate",
        "Tipo_de_la_cita__c",
        "AccountId",
        "Clinica__c",
        "Empresa__c",
        "Empresa_Principal__c",
    ]
]
Citas_de_servicio["AppointmentNumber"] = "F" + Citas_de_servicio["AppointmentNumber"]

Citas_de_servicio.head()

In [ ]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber,Empresa__c,Empresa_Principal__c from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2["Clinica__c"] = "Orientación Médica"
Citas_de_servicio2["Tipo_de_la_cita__c"] = "Agendamiento"
# Citas_de_servicio2["Empresas"] = np.nan
# Citas_de_servicio2["Empresas Id"] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",
        "RUT__c",
        "CreatedDate",
        "Tipo_de_la_cita__c",
        "AccountId",
        "Clinica__c",
        "Empresa__c",
        "Empresa_Principal__c",
    ]
]
Citas_de_servicio2.head(10)

In [ ]:
# unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "AccountId": "Id Paciente",
        "Clinica__c": "Clínica",
        "Empresa__c": "Empresas",
        "Empresa_Principal__c": "Id Empresa Account",
    }
)

Gran_Citas_deservicio

In [ ]:
# Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c,Pais_Paciente__c,Empresa_Principal__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'Empresa_Principal__c',
    ]
]
ficha_deporte["clinica"] = "Deporte"
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        'Empresa_Principal__c': 'Empresa Principal'
    }
)
ficha_deporte.shape

In [ ]:
ficha_deporte
